In [2]:
# !pip install langchain-openai

In [3]:
import pandas as pd
import json
from langchain_openai import AzureChatOpenAI
from datetime import datetime

In [4]:
creds_path = r"C:\Pranav\azure_openai_creds.json"

with open(creds_path, "r") as f:
    creds = json.load(f)

llm = AzureChatOpenAI(
    azure_endpoint=creds["azure_endpoint"],
    api_key=creds["OPENAI_API_KEY"],
    api_version=creds["OPENAI_API_VERSION"],
    deployment_name=creds["deployment_name"]
)

In [5]:
data_1 = [
    {
        "scenario": "Valid Vehicle",
        "description": "A policyholder submits a claim for vehicle damage after a rear-end collision, providing a police report, photos, repair estimates, and proof of insurance.",
        "claim_type": "Auto Insurance (Collision Coverage)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid up to the incident date.",
            "covered_driver: The driver must be listed on the policy or fall under permissive use clauses.",
            "incident_coverage: The collision must be covered under the policy’s collision terms (e.g., not excluded due to intentional acts).",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., within 7 days).",
            "documentation_completeness: The claim must include police report, photos, repair estimates, and witness statements (if applicable).",
            "deductible_application: The policyholder must pay the applicable deductible per policy terms.",
            "no_fraud_indicators: The claim must show no signs of fraud, such as inconsistent statements.",
            "damage_verification: The reported damage must align with the incident and be verified by an adjuster or repair estimate."
        ]
    },
    {
        "scenario": "Missing Document",
        "description": "A policyholder submits a claim for storm damage to their home, including photos and a repair estimate, but omits the proof of loss form and weather report.",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "covered_rules": [
            "policy_active_status: The policy must be active with no lapse in premium payments.",
            "covered_peril: The damage must result from a covered peril (e.g., windstorm) and not an excluded event (e.g., flood).",
            "proof_of_loss: The policyholder must submit a proof of loss form within the specified period (e.g., 60 days).",
            "mitigation_efforts: The policyholder must have taken steps to prevent further damage (e.g., covering the damaged area).",
            "documentation_completeness: The claim must include photos, repair estimates, and evidence of the storm (e.g., weather reports).",
            "actual_cash_value_or_replacement_cost: The settlement must align with the policy’s valuation method, minus deductible.",
            "no_wear_and_tear: The damage must not be due to pre-existing wear or lack of maintenance.",
            "compliance_with_local_codes: Repairs must comply with local building codes, with coverage limits for code upgrades."
        ]
    },
    {
        "scenario": "Duplicate Bill",
        "description": "A policyholder submits a claim for hospital stay, including two identical hospital bills for the same service, along with medical records and proof of payment.",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid through the date of service.",
            "covered_service: The treatment must be medically necessary and not excluded (e.g., not cosmetic).",
            "in_network_provider: The hospital must be in-network, or out-of-network coverage must apply with higher cost-sharing.",
            "timely_submission: The claim must be submitted within the policy’s timeframe (e.g., 90 days).",
            "documentation_completeness: The claim must include itemized bills, medical records, and proof of payment (if applicable).",
            "duplicate_bill_check: The claim must not include duplicate bills for the same service, verified against billing codes and dates.",
            "co_pay_or_co_insurance: The policyholder must meet co-pay or co-insurance obligations per the policy.",
            "no_pre_existing_condition_exclusion: The condition must not be excluded as a pre-existing condition."
        ]
    },
    {
        "scenario": "Mid-level Fraud",
        "description": "A policyholder submits a claim for a car accident, but the repair estimate includes damages inconsistent with the police report, suggesting exaggerated claims.",
        "claim_type": "Auto Insurance (Collision Coverage)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid up to the incident date.",
            "covered_driver: The driver must be listed on the policy or fall under permissive use clauses.",
            "incident_coverage: The collision must be covered under the policy’s collision terms.",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., within 7 days).",
            "documentation_completeness: The claim must include police report, photos, repair estimates, and witness statements (if applicable).",
            "fraud_detection: The claim must show no signs of fraud, such as exaggerated damages or inconsistencies between reports and estimates.",
            "deductible_application: The policyholder must pay the applicable deductible per policy terms.",
            "damage_verification: The reported damage must align with the incident and be verified by an adjuster."
        ]
    },
    {
        "scenario": "Valid Medical",
        "description": "A policyholder submits a claim for a medically necessary knee surgery, including hospital bills, surgeon’s fees, medical records, and proof of in-network provider.",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid through the date of service.",
            "covered_service: The surgery must be medically necessary and not excluded (e.g., not experimental).",
            "in_network_provider: The hospital and surgeon must be in-network, or out-of-network coverage must apply.",
            "pre_authorization: Pre-authorization must be obtained for non-emergency procedures (if required).",
            "timely_submission: The claim must be submitted within the policy’s timeframe (e.g., 90 days).",
            "documentation_completeness: The claim must include itemized bills, medical records, and proof of payment (if applicable).",
            "co_pay_or_co_insurance: The policyholder must meet co-pay or co-insurance obligations per the policy.",
            "no_pre_existing_condition_exclusion: The condition must not be excluded as a pre-existing condition."
        ]
    },
    {
        "scenario": "High Fraud",
        "description": "A claimant submits a life insurance claim for a policyholder’s death, but the death certificate appears altered, and the policy application contains misrepresentations.",
        "claim_type": "Term Life Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be in force at the time of death with all premiums paid.",
            "covered_cause_of_death: The death must not fall under exclusions (e.g., suicide within the contestability period).",
            "beneficiary_eligibility: The claimant must be the named beneficiary or have legal rights to the proceeds.",
            "timely_notification: The insurer must be notified within the policy’s specified period (e.g., 30 days).",
            "documentation_completeness: The claim must include a certified death certificate, policy documents, and proof of beneficiary identity.",
            "fraud_detection: The claim must show no signs of fraud, such as altered documents or misrepresentations in the policy application.",
            "contestability_period: If within the contestability period, the insurer may review for material misrepresentations.",
            "no_fraudulent_application: The policy application must not contain fraudulent misstatements."
        ]
    },
    {
        "scenario": "Policy Lapse",
        "description": "A policyholder submits a claim for fire damage to their business, but the policy lapsed due to unpaid premiums before the incident date.",
        "claim_type": "Commercial Property Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid up to the incident date.",
            "covered_peril: Fire must be a covered peril, not excluded (e.g., not arson by the insured).",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., 72 hours).",
            "documentation_completeness: The claim must include fire department reports, photos, inventory lists, and repair estimates.",
            "mitigation_efforts: The insured must have mitigated further damage (e.g., securing the property post-fire).",
            "business_interruption_coverage: If claimed, business interruption losses must be documented and align with policy limits.",
            "valuation_method: Losses must be calculated per the policy’s terms (e.g., replacement cost or actual cash value).",
            "compliance_with_safety_standards: The property must have complied with fire safety codes at the time of the incident."
        ]
    }
]

data_2 = [
    {
        "scenario": "Stolen Vehicle Claim",
        "description": "A policyholder submits a claim for a stolen car, providing a police report, vehicle registration, and proof of ownership, but no proof of anti-theft device usage.",
        "claim_type": "Auto Insurance (Comprehensive Coverage)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid up to the incident date.",
            "covered_incident: Theft must be covered under the policy’s comprehensive terms.",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., within 7 days).",
            "documentation_completeness: The claim must include police report, vehicle registration, and proof of ownership.",
            "anti_theft_device_compliance: If required, the vehicle must have had an active anti-theft device as per policy terms.",
            "no_fraud_indicators: The claim must show no signs of fraud, such as inconsistent police report details.",
            "loss_verification: The theft must be verified by police records and insurer investigation.",
            "deductible_application: The policyholder must pay the applicable deductible per policy terms."
        ]
    },
    {
        "scenario": "Water Damage Claim",
        "description": "A homeowner submits a claim for water damage due to a burst pipe, including photos, a plumber’s report, and repair estimates, but lacks proof of mitigation efforts.",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "covered_rules": [
            "policy_active_status: The policy must be active with no lapse in premium payments.",
            "covered_peril: Water damage from a burst pipe must be covered, not excluded (e.g., not due to flooding).",
            "proof_of_loss: The policyholder must submit a proof of loss form within the specified period (e.g., 60 days).",
            "mitigation_efforts: The policyholder must have taken steps to prevent further damage (e.g., shutting off water supply).",
            "documentation_completeness: The claim must include photos, plumber’s report, and repair estimates.",
            "actual_cash_value_or_replacement_cost: The settlement must align with the policy’s valuation method, minus deductible.",
            "no_maintenance_neglect: The damage must not result from lack of maintenance or pre-existing issues.",
            "compliance_with_local_codes: Repairs must comply with local plumbing and building codes."
        ]
    },
    {
        "scenario": "Outpatient Procedure Claim",
        "description": "A policyholder submits a claim for an outpatient colonoscopy, including medical bills and records, but the procedure was performed by an out-of-network provider.",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid through the date of service.",
            "covered_service: The procedure must be medically necessary and not excluded (e.g., not cosmetic).",
            "in_network_provider: The provider must be in-network, or out-of-network coverage must apply with higher cost-sharing.",
            "pre_authorization: Pre-authorization must be obtained for non-emergency procedures (if required).",
            "timely_submission: The claim must be submitted within the policy’s timeframe (e.g., 90 days).",
            "documentation_completeness: The claim must include itemized bills, medical records, and proof of payment (if applicable).",
            "co_pay_or_co_insurance: The policyholder must meet co-pay or co-insurance obligations per the policy.",
            "no_pre_existing_condition_exclusion: The condition must not be excluded as a pre-existing condition."
        ]
    },
    {
        "scenario": "Suspicious Injury Claim",
        "description": "A policyholder submits a claim for a workplace injury under a workers’ compensation policy, but the incident report conflicts with witness statements.",
        "claim_type": "Workers’ Compensation Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid by the employer.",
            "covered_incident: The injury must have occurred during work-related activities and be covered under the policy.",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., 30 days).",
            "documentation_completeness: The claim must include incident report, medical records, and witness statements.",
            "fraud_detection: The claim must show no signs of fraud, such as conflicting reports or staged injuries.",
            "medical_verification: The injury must be verified by medical records and a qualified physician.",
            "compliance_with_safety_regulations: The workplace must comply with applicable safety regulations.",
            "benefit_eligibility: The policyholder must be eligible for benefits under the policy terms."
        ]
    },
    {
        "scenario": "Travel Insurance Trip Cancellation",
        "description": "A policyholder submits a claim for trip cancellation due to a medical emergency, providing medical records and travel receipts, but no proof of non-refunded expenses.",
        "claim_type": "Travel Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be active and purchased before the cancellation event.",
            "covered_reason: The cancellation must be due to a covered reason (e.g., medical emergency, not personal choice).",
            "timely_notification: The insurer must be notified within the policy’s timeframe (e.g., 48 hours).",
            "documentation_completeness: The claim must include medical records, travel receipts, and proof of non-refunded expenses.",
            "no_pre_existing_condition_exclusion: The medical emergency must not be due to a pre-existing condition, unless covered.",
            "verification_of_emergency: The medical emergency must be verified by a licensed physician.",
            "policy_limits: The reimbursement must not exceed the policy’s coverage limits for trip cancellation.",
            "no_fraud_indicators: The claim must show no signs of fraud, such as falsified medical records."
        ]
    },
    {
        "scenario": "Vandalism Property Claim",
        "description": "A business owner submits a claim for vandalism damage to their store’s windows, including a police report and repair estimates, but the incident occurred after hours.",
        "claim_type": "Commercial Property Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid up to the incident date.",
            "covered_peril: Vandalism must be a covered peril, not excluded under policy terms.",
            "timely_reporting: The claim must be reported within the policy’s timeframe (e.g., 72 hours).",
            "documentation_completeness: The claim must include police report, photos, and repair estimates.",
            "mitigation_efforts: The insured must have mitigated further damage (e.g., boarding up windows).",
            "no_fraud_indicators: The claim must show no signs of fraud, such as insider involvement.",
            "valuation_method: Losses must be calculated per the policy’s terms (e.g., replacement cost).",
            "security_measures_compliance: The property must have had reasonable security measures in place."
        ]
    },
    {
        "scenario": "Invalid Beneficiary Claim",
        "description": "A claimant submits a life insurance claim, but they are not the named beneficiary, providing a death certificate and policy documents.",
        "claim_type": "Term Life Insurance",
        "covered_rules": [
            "policy_active_status: The policy must be in force at the time of death with all premiums paid.",
            "covered_cause_of_death: The death must not fall under exclusions (e.g., suicide within contestability period).",
            "beneficiary_eligibility: The claimant must be the named beneficiary or have legal rights to the proceeds.",
            "timely_notification: The insurer must be notified within the policy’s specified period (e.g., 30 days).",
            "documentation_completeness: The claim must include a certified death certificate, policy documents, and proof of beneficiary identity.",
            "contestability_period: If within the contestability period, the insurer may review for misrepresentations.",
            "no_fraudulent_application: The policy application must not contain fraudulent misstatements.",
            "payout_method: The payout must align with the policy’s terms (e.g., lump sum or annuity)."
        ]
    },
    {
        "scenario": "Incorrect Repair Estimate",
        "description": "A policyholder submits a claim for home damage from a fallen tree, but the repair estimate includes unrelated upgrades not covered by the policy.",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "covered_rules": [
            "policy_active_status: The policy must be active with no lapse in premium payments.",
            "covered_peril: The damage must result from a covered peril (e.g., fallen tree due to windstorm).",
            "proof_of_loss: The policyholder must submit a proof of loss form within the specified period (e.g., 60 days).",
            "documentation_completeness: The claim must include photos, repair estimates, and evidence of the incident.",
            "covered_repairs_only: The claim must cover only damages directly caused by the covered peril, not unrelated upgrades.",
            "mitigation_efforts: The policyholder must have taken steps to prevent further damage.",
            "actual_cash_value_or_replacement_cost: The settlement must align with the policy’s valuation method.",
            "no_fraud_indicators: The claim must show no signs of fraud, such as inflated estimates."
        ]
    },
    {
        "scenario": "Expired Coverage Claim",
        "description": "A policyholder submits a claim for a medical procedure, but the policy expired one month before the date of service.",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "covered_rules": [
            "policy_active_status: The policy must be active with premiums paid through the date of service.",
            "covered_service: The procedure must be medically necessary and not excluded.",
            "in_network_provider: The provider must be in-network, or out-of-network coverage must apply.",
            "pre_authorization: Pre-authorization must be obtained for non-emergency procedures (if required).",
            "timely_submission: The claim must be submitted within the policy’s timeframe (e.g., 90 days).",
            "documentation_completeness: The claim must include itemized bills, medical records, and proof of payment (if applicable).",
            "co_pay_or_co_insurance: The policyholder must meet co-pay or co-insurance obligations per the policy.",
            "no_pre_existing_condition_exclusion: The condition must not be excluded as a pre-existing condition."
        ]
    },
    {
        "scenario": "Flood Damage Claim",
        "description": "A homeowner submits a claim for flood damage to their basement, but their standard homeowners policy excludes flood coverage.",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "covered_rules": [
            "policy_active_status: The policy must be active with no lapse in premium payments.",
            "covered_peril: The damage must result from a covered peril; flood is typically excluded unless separate flood insurance exists.",
            "proof_of_loss: The policyholder must submit a proof of loss form within the specified period (e.g., 60 days).",
            "documentation_completeness: The claim must include photos, repair estimates, and evidence of the flood.",
            "mitigation_efforts: The policyholder must have taken steps to prevent further damage (e.g., removing water).",
            "actual_cash_value_or_replacement_cost: The settlement must align with the policy’s valuation method, if covered.",
            "no_maintenance_neglect: The damage must not result from lack of maintenance.",
            "flood_coverage_check: The policy must explicitly include flood coverage for the claim to be valid."
        ]
    }
]

data = []
data.extend(data_1)
data.extend(data_2)

df_scenarios = pd.DataFrame(data)

In [6]:
claims = [
    # Valid Claims
    {
        "name": "Valid Vehicle",
        "policy_id": "VHC-1001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Auto Insurance (Collision Coverage)",
        "accident_date": "2025-06-20",
        "claim_amount": 5000,
        "document": "repair_bill_vehicle.pdf",
        "is_valid_claim": True,
        "reason": "All required documents provided, policy active, damage covered under collision terms, no fraud detected."
    },
    {
        "name": "Valid Medical",
        "policy_id": "HLT-2001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "accident_date": "2025-05-15",
        "claim_amount": 12000,
        "document": "hospital_bill_knee_surgery.pdf",
        "is_valid_claim": True,
        "reason": "Medically necessary procedure, in-network provider, complete documentation, policy active."
    },
    {
        "name": "Stolen Vehicle Claim",
        "policy_id": "VHC-1002",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Auto Insurance (Comprehensive Coverage)",
        "accident_date": "2025-06-05",
        "claim_amount": 25000,
        "document": "police_report_theft.pdf",
        "is_valid_claim": True,
        "reason": "Theft covered under comprehensive terms, police report and ownership proof provided, policy active."
    },
    {
        "name": "Water Damage Claim",
        "policy_id": "HOM-3002",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "accident_date": "2025-03-12",
        "claim_amount": 6000,
        "document": "plumber_report.pdf",
        "is_valid_claim": True,
        "reason": "Burst pipe covered, complete documentation including mitigation efforts, policy active."
    },
    {
        "name": "Outpatient Procedure Claim",
        "policy_id": "HLT-2002",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "accident_date": "2025-06-10",
        "claim_amount": 3000,
        "document": "colonoscopy_bill.pdf",
        "is_valid_claim": True,
        "reason": "Medically necessary procedure, in-network provider, timely submission, policy active."
    },
    {
        "name": "Travel Insurance Trip Cancellation",
        "policy_id": "TRV-5001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Travel Insurance",
        "accident_date": "2025-05-28",
        "claim_amount": 4000,
        "document": "medical_emergency_letter.pdf",
        "is_valid_claim": True,
        "reason": "Covered medical emergency, non-refunded expenses documented, policy active."
    },
    {
        "name": "Vandalism Property Claim",
        "policy_id": "COM-6001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Commercial Property Insurance",
        "accident_date": "2025-06-15",
        "claim_amount": 3500,
        "document": "police_report_vandalism.pdf",
        "is_valid_claim": True,
        "reason": "Vandalism covered, police report and repair estimates provided, policy active."
    },
    {
        "name": "Home Windstorm Claim",
        "policy_id": "HOM-3001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "accident_date": "2025-04-25",
        "claim_amount": 15000,
        "document": "roof_repair_estimate.png",
        "is_valid_claim": True,
        "reason": "Covered peril (windstorm), proof of loss submitted, mitigation efforts documented, policy active."
    },
    {
        "name": "Workplace Injury Claim",
        "policy_id": "WCP-4001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Workers’ Compensation Insurance",
        "accident_date": "2025-02-20",
        "claim_amount": 15000,
        "document": "incident_report.pdf",
        "is_valid_claim": True,
        "reason": "Work-related injury verified, complete documentation, no fraud, policy active."
    },
    {
        "name": "Life Insurance Payout",
        "policy_id": "LIF-7001",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Term Life Insurance",
        "accident_date": "2025-04-18",
        "claim_amount": 100000,
        "document": "death_certificate.pdf",
        "is_valid_claim": True,
        "reason": "Named beneficiary, certified death certificate, no exclusions, policy active."
    },
    # Invalid Claims
    {
        "name": "Missing Document",
        "policy_id": "HOM-3003",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "accident_date": "2025-06-18",
        "claim_amount": 7000,
        "document": "repair_estimate.png",
        "is_valid_claim": False,
        "reason": "Missing proof of loss form required for claim validation."
    },
    {
        "name": "Duplicate Bill",
        "policy_id": "HLT-2003",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "accident_date": "2025-05-12",
        "claim_amount": 10000,
        "document": "hospital_bill_duplicate.pdf",
        "is_valid_claim": False,
        "reason": "Duplicate bills submitted for the same service."
    },
    {
        "name": "Mid-level Fraud",
        "policy_id": "VHC-1003",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Auto Insurance (Collision Coverage)",
        "accident_date": "2025-06-09",
        "claim_amount": 15000,
        "document": "repair_estimate_exaggerated.pdf",
        "is_valid_claim": False,
        "reason": "Exaggerated damages in repair estimate inconsistent with police report."
    },
    {
        "name": "High Fraud",
        "policy_id": "LIF-7002",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Term Life Insurance",
        "accident_date": "2025-03-10",
        "claim_amount": 200000,
        "document": "altered_death_certificate.pdf",
        "is_valid_claim": False,
        "reason": "Altered death certificate detected, indicating fraudulent claim."
    },
    {
        "name": "Policy Lapse",
        "policy_id": "COM-6002",
        "policy_start": "2024-01-01",
        "policy_end": "2024-12-31",
        "claim_type": "Commercial Property Insurance",
        "accident_date": "2025-01-08",
        "claim_amount": 50000,
        "document": "fire_department_report.pdf",
        "is_valid_claim": False,
        "reason": "Policy lapsed due to unpaid premiums before incident date."
    },
    {
        "name": "Invalid Beneficiary Claim",
        "policy_id": "LIF-7003",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Term Life Insurance",
        "accident_date": "2025-06-03",
        "claim_amount": 150000,
        "document": "death_certificate.pdf",
        "is_valid_claim": False,
        "reason": "Claimant is not the named beneficiary or legally entitled to proceeds."
    },
    {
        "name": "Incorrect Repair Estimate",
        "policy_id": "HOM-3004",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "accident_date": "2025-04-12",
        "claim_amount": 10000,
        "document": "repair_estimate_upgrades.pdf",
        "is_valid_claim": False,
        "reason": "Repair estimate includes unrelated upgrades not covered by policy."
    },
    {
        "name": "Expired Coverage Claim",
        "policy_id": "HLT-2004",
        "policy_start": "2024-01-01",
        "policy_end": "2024-12-31",
        "claim_type": "Health Insurance (Major Medical Plan)",
        "accident_date": "2025-01-22",
        "claim_amount": 8000,
        "document": "medical_bill.pdf",
        "is_valid_claim": False,
        "reason": "Policy expired before the date of medical service."
    },
    {
        "name": "Flood Damage Claim",
        "policy_id": "HOM-3005",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Homeowners Insurance (HO-3 Policy)",
        "accident_date": "2025-06-22",
        "claim_amount": 20000,
        "document": "flood_damage_photos.png",
        "is_valid_claim": False,
        "reason": "Flood damage not covered under standard HO-3 policy."
    },
    {
        "name": "Suspicious Injury Claim",
        "policy_id": "WCP-4002",
        "policy_start": "2025-01-01",
        "policy_end": "2025-12-31",
        "claim_type": "Workers’ Compensation Insurance",
        "accident_date": "2025-05-27",
        "claim_amount": 20000,
        "document": "incident_report_conflicting.pdf",
        "is_valid_claim": False,
        "reason": "Conflicting witness statements indicate potential staged injury."
    }
]

df_claims = pd.DataFrame(claims)

In [7]:
df_scenarios = df_scenarios.drop(columns="covered_rules")

In [8]:
df_scenarios.head()

scenario                                        description  \
0     Valid Vehicle  A policyholder submits a claim for vehicle dam...   
1  Missing Document  A policyholder submits a claim for storm damag...   
2    Duplicate Bill  A policyholder submits a claim for hospital st...   
3   Mid-level Fraud  A policyholder submits a claim for a car accid...   
4     Valid Medical  A policyholder submits a claim for a medically...   

                              claim_type  
0    Auto Insurance (Collision Coverage)  
1     Homeowners Insurance (HO-3 Policy)  
2  Health Insurance (Major Medical Plan)  
3    Auto Insurance (Collision Coverage)  
4  Health Insurance (Major Medical Plan)

In [9]:
df_scenarios["description"][3]

'A policyholder submits a claim for a car accident, but the repair estimate includes damages inconsistent with the police report, suggesting exaggerated claims.'

In [10]:
# temp = {
#         "name": "Valid Vehicle Damage",
#         "policy_id": "VHC-1001",
#         "policy_start": "2025-01-01",
#         "policy_end": "2025-12-31",
#         "claim_type": "Vehicle",
#         "accident_date": "2025-06-12",
#         "claim_amount": 5000,
#         "document": "repair_bill_vehicle.png",
#         "is_valid_claim": True
#     }

# print(temp.keys())

check category of mid level is a fraud or just inconsistency

In [11]:
# df_temp = df_scenarios[["claim_type","covered_rules"]].groupby("claim_type").agg(list).reset_index()

# for ind, row in df_temp.iterrows():

#   print(row["claim_type"])
#   for i in row["covered_rules"]:
#     print(i)

In [13]:
df_claim_type_wise_metadata = df_scenarios.groupby("claim_type").apply(lambda x: dict(zip(x["scenario"], x["description"]))).reset_index().rename(columns={0:"scenario_and_description"})

C:\Users\UTHRAVFST\AppData\Local\Temp\ipykernel_50108\1338167722.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_claim_type_wise_metadata = df_scenarios.groupby("claim_type").apply(lambda x: dict(zip(x["scenario"], x["description"]))).reset_index().rename(columns={0:"scenario_and_description"})


In [14]:
df_claim_type_wise_metadata.head(2)

claim_type  \
0      Auto Insurance (Collision Coverage)   
1  Auto Insurance (Comprehensive Coverage)   

                            scenario_and_description  
0  {'Valid Vehicle': 'A policyholder submits a cl...  
1  {'Stolen Vehicle Claim': 'A policyholder submi...

In [16]:
# metadata = df_claim_type_wise_metadata.to_dict("records")[0]

# prompt = f"""
# You are an insurance claim processing expert tasked with generating common covered rules for the claim type {metadata['claim_type']} based on the provided scenarios and descriptions. The rules should be tailored for three agents: a document verification agent, an eligibility checker agent, and a fraud checker agent. Each agent requires a list of rules to evaluate claims of this type. Output the results in a JSON format, with keys for each agent (document_verification_agent, eligibility_checker_agent, fraud_checker_agent) and values as lists of rule strings. Ensure the output is strictly JSON and contains no additional text, no triple backticks, and no Markdown formatting.

# ### Claim Type:
# {metadata['claim_type']}

# ### Scenarios and Descriptions:
# {''.join([f'- {scenario}: {description} ' for scenario, description in metadata['scenario_and_description'].items()])}

# ### Instructions:
# 1. Generate a list of common covered rules for each agent based on the claim type and scenarios provided.
# 2. Rules should be concise, specific to the agent's role, and formatted as strings (e.g., 'policy_active_status: The policy must be active with premiums paid.').
# 3. The document verification agent focuses on checking the completeness and authenticity of submitted documents.
# 4. The eligibility checker agent focuses on verifying policy coverage, incident eligibility, and compliance with policy terms.
# 5. The fraud checker agent focuses on detecting inconsistencies, exaggerations, or fraudulent activities in the claim.
# 6. Output only the JSON object with the following structure, without triple backticks or any other formatting:

# {{
#   "document_verification_agent": ["<rule 1>", "<rule 2>", ...],
#   "eligibility_checker_agent": ["<rule 1>", "<rule 2>", ...],
#   "fraud_checker_agent": ["<rule 1>", "<rule 2>", ...]
# }}

# ### Notes:
# - Use the scenarios to infer typical requirements and potential issues (e.g., inconsistent repair estimates in the Mid-level Fraud scenario).
# - Ensure rules are general enough to apply to most Auto Insurance (Collision Coverage) claims but specific to the agent's role.
# - Return only parseable JSON, with no additional text, comments, or formatting like triple backticks.
# """

# # Call AzureChatOpenAI with the prompt
# try:
#     response = llm.invoke(prompt)
#     # Parse and print the JSON response
#     try:
#         validation_results = json.loads(response.content)
#         print(json.dumps(validation_results, indent=4))
#     except json.JSONDecodeError:
#         print(f"Error: LLM response is not valid JSON. Raw response:\n{response.content}")
# except Exception as e:
#     print(f"Error invoking LLM: {str(e)}")


In [18]:
df_claim_type_wise_metadata["covered_rules"] = None

In [21]:
for ind, row in df_claim_type_wise_metadata.iterrows():
    
    metadata = row

    prompt = f"""
    You are an insurance claim processing expert tasked with generating common covered rules for the claim type {metadata['claim_type']} based on the provided scenarios and descriptions. The rules should be tailored for three agents: a document verification agent, an eligibility checker agent, and a fraud checker agent. Each agent requires a list of rules to evaluate claims of this type. Output the results in a JSON format, with keys for each agent (document_verification_agent, eligibility_checker_agent, fraud_checker_agent) and values as lists of rule strings. Ensure the output is strictly JSON and contains no additional text, no triple backticks, and no Markdown formatting.

    ### Claim Type:
    {metadata['claim_type']}

    ### Scenarios and Descriptions:
    {''.join([f'- {scenario}: {description} ' for scenario, description in metadata['scenario_and_description'].items()])}

    ### Instructions:
    1. Generate a list of common covered rules for each agent based on the claim type and scenarios provided.
    2. Rules should be concise, specific to the agent's role, and formatted as strings (e.g., 'policy_active_status: The policy must be active with premiums paid.').
    3. The document verification agent focuses on checking the completeness and authenticity of submitted documents.
    4. The eligibility checker agent focuses on verifying policy coverage, incident eligibility, and compliance with policy terms.
    5. The fraud checker agent focuses on detecting inconsistencies, exaggerations, or fraudulent activities in the claim.
    6. Output only the JSON object with the following structure, without triple backticks or any other formatting:

    {{
    "document_verification_agent": ["<rule 1>", "<rule 2>", ...],
    "eligibility_checker_agent": ["<rule 1>", "<rule 2>", ...],
    "fraud_checker_agent": ["<rule 1>", "<rule 2>", ...]
    }}

    ### Notes:
    - Use the scenarios to infer typical requirements and potential issues (e.g., inconsistent repair estimates in the Mid-level Fraud scenario).
    - Ensure rules are general enough to apply to most Auto Insurance (Collision Coverage) claims but specific to the agent's role.
    - Return only parseable JSON, with no additional text, comments, or formatting like triple backticks.
    """

    # Call AzureChatOpenAI with the prompt
    try:
        response = llm.invoke(prompt)
        # Parse and print the JSON response
        try:
            validation_results = json.loads(response.content)
            df_claim_type_wise_metadata.at[ind, "covered_rules"] = validation_results  # Use .at for dictionary assignment
            # print(json.dumps(validation_results, indent=4))
        except json.JSONDecodeError:
            print(f"Error: LLM response is not valid JSON. Raw response:\n{response.content}")
    except Exception as e:
        print(f"Error invoking LLM: {str(e)}")

try to generate documents using LLM

In [31]:
df_scenarios = df_scenarios.merge(df_claim_type_wise_metadata[["claim_type","covered_rules"]], on="claim_type", how="left")

In [32]:
df_scenarios.head(2)

scenario                                        description  \
0     Valid Vehicle  A policyholder submits a claim for vehicle dam...   
1  Missing Document  A policyholder submits a claim for storm damag...   

                            claim_type  \
0  Auto Insurance (Collision Coverage)   
1   Homeowners Insurance (HO-3 Policy)   

                                       covered_rules  
0  {'document_verification_agent': ['police_repor...  
1  {'document_verification_agent': ['proof_of_los...

In [39]:
for ind, row in df_scenarios.iterrows():

    print(row["scenario"])
    print(row["claim_type"])
    print(row["description"])
    print(row["covered_rules"])

Valid Vehicle
Auto Insurance (Collision Coverage)
A policyholder submits a claim for vehicle damage after a rear-end collision, providing a police report, photos, repair estimates, and proof of insurance.
{'document_verification_agent': ['police_report_validity: The police report must be complete and legible.', 'photos_authenticity: Photographic evidence of the damage must be recent and consistent with the reported incident.', 'repair_estimate_validity: Repair estimates must be provided with details from recognized service providers.', "proof_of_insurance_validity: The submitted proof of insurance must match the policyholder's information and be current."], 'eligibility_checker_agent': ['policy_active_status: The policy must be active with all premiums paid up to date at the time of the incident.', 'coverage_compliance: The damages claimed must fall under collision coverage as outlined in the policy terms.', 'incident_date_verification: The incident must occur within the covered policy

In [33]:
df_claims.head(2)

name policy_id policy_start  policy_end  \
0  Valid Vehicle  VHC-1001   2025-01-01  2025-12-31   
1  Valid Medical  HLT-2001   2025-01-01  2025-12-31   

                              claim_type accident_date  claim_amount  \
0    Auto Insurance (Collision Coverage)    2025-06-20          5000   
1  Health Insurance (Major Medical Plan)    2025-05-15         12000   

                         document  is_valid_claim  \
0         repair_bill_vehicle.pdf            True   
1  hospital_bill_knee_surgery.pdf            True   

                                              reason  
0  All required documents provided, policy active...  
1  Medically necessary procedure, in-network prov...

In [34]:
df_scenarios.to_csv("df_scenarios.csv", index=False)
df_claims.to_csv("df_claims.csv", index=False)

In [54]:
# df_claims = df_claims.merge(df_scenarios, left_on=["name","claim_type"], right_on=["scenario","claim_type"], how="left")

aggregate all fraud related rules should be aggregated for vehicle, medical, add notes in above df.
there only be only 1 list of vehicle related fraud rules, another list of medical related fraud rules